In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# путь к БД на  компьютере
path_to_db_local = 'C:/Users/paman/OneDrive/Рабочий стол/Стася/YP/telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

In [3]:
# если путь на компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

In [5]:
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    
    # пример запроса
    query = """
                SELECT user.user_id,
                       user.lt_day,
        
                       CASE
                            WHEN user.lt_day<=365 THEN 'new'
                            ELSE 'old'
                       END as is_new,
                       
                       user.age,
                       user.os_name,
                       
                       CASE
                            WHEN user.gender_segment=1 THEN 'женщина'
                            WHEN user.gender_segment=0 THEN 'мужчина'
                            ELSE 'none'
                       END as gender_segment,
                       
                        user.cpe_type_name,
                        location.country,
                        location.city,
                        age_segment.title as age_segment,
                        traffic_segment.title as traffic_segment,
                        lifetime_segment.title as lifetime_segment,
                        user.nps_score,
                        
                        CASE
                            WHEN user.nps_score >=0 and user.nps_score<=6 THEN 'критики'
                            WHEN user.nps_score >=7 and user.nps_score<=8 THEN 'нейтральные'
                            ELSE 'сторонники'
                        END as npc_group
        
                 FROM user
                 JOIN location on user.location_id=location.location_id
                 JOIN age_segment on user.age_gr_id=age_segment.age_gr_id
                 JOIN traffic_segment on user.tr_gr_id=traffic_segment.tr_gr_id
                 JOIN lifetime_segment on user.lt_gr_id=lifetime_segment.lt_gr_id
                
    """
    
    # создаём датафрейм по данным запроса
    df = pd.read_sql(query, engine) 

In [6]:
df

,user_id,lt_day,is_new,age,os_name,gender_segment,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,npc_group
0,A001A2,2320,old,45.0,ANDROID,женщина,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,old,53.0,ANDROID,мужчина,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,old,57.0,ANDROID,мужчина,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники
3,A004TB,4190,old,44.0,IOS,женщина,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,сторонники
4,A004XT,1163,old,24.0,ANDROID,мужчина,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,сторонники
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,old,54.0,ANDROID,женщина,SMARTPHONE,Россия,Москва,05 45-54,04 1-5,08 36+,5,критики
502489,ZZZLWY,129,new,31.0,ANDROID,мужчина,SMARTPHONE,Россия,Москва,03 25-34,04 1-5,04 4-6,8,нейтральные
502490,ZZZQ5F,522,old,36.0,ANDROID,мужчина,SMARTPHONE,Россия,Сургут,04 35-44,09 25-30,06 13-24,10,сторонники
502491,ZZZQ8E,2936,old,37.0,ANDROID,женщина,SMARTPHONE,Россия,УланУдэ,04 35-44,17 65-70,08 36+,9,сторонники


In [7]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

преобразовали данные в csv формат, файл появился в папке с ноутбуком, теперь данный файл можно загрузить в Tableau.

**Ссылка на дашборд:**

https://public.tableau.com/app/profile/stasy2969/viz/2__16782639947330/Dashboard1?publish=yes

**Ссылка на презентацию:**

https://disk.yandex.ru/d/nJWLAzsWO9P1qg